# Assignment - 2: Data Representation and Point Cloud Operations

Team Name: \<team name here\> 

Roll number: \<Roll number here (in sorted order)\>

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 26/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.

# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Introduction to types of Transformations and Homogeneous coordinates

In robotics applications, it is inevitable to keep track of the frames of multiple objects/worlds. These frames can be transformations from one coordinate frame to the other. **Homogeneous coordinates** help in keeping track of various coordinate frames and allow performing composition of various transforms. We will first try to understand between types of transformations and their invariant properties.
1. What is the difference between Affine, Similarity, and Euclidean transform? What are the invariant properities of each type of transform?
2. Watch this [video](https://www.youtube.com/watch?v=PvEl63t-opM) to briefly understand homogeneous coordinates. What are points at infinity? What type of transformation can you apply to transform a point from infinity to a point that is not at infinity? 
3. Using homogeneous coordinates we can represent different types of transformation as point transforms vs. frame transforms. Concatenation of transforms (whether you post multiply transformation matrices or pre-multiply transformation matrices) depends on the problem and how you are viewing it. Try to understand the difference between frame vs. point transformations from this [video](https://youtu.be/Za7Sdegf8m8?t=1834). Let's assume that our camera and world frames are coinciding with each other. We need to estimate the camera to world **frame** transformation matrix after applying the transformations defined below in terms of $T_i$.We apply **frame** transform to move the camera in the world in the following order:
    1. $T_1$ from the camera coordinate frame.
    2. $T_2$ from the world coordinate frame.
    3. $T_3$ from the world coordinate frame.
    4. $T_4$ from the camera coordinate frame.
    5. $T_5$ from the camera coordinate frame.


## Solution

3. We want camera to world *frame* transformation matrix  = $T^C_W$ <br>

Initialy, camera and world frame coincide, $T_{initial} = I$
- $T_1$ *frame* transformation from camera frame $ T^W_{C_A} = I.T_1 $
- $T_2$ *frame* transformation from world frame $ T^W_{C_B} = T_2.I.T_1 $
- $T_3$ *frame* transformation from world frame $ T^W_{C_C} = T_3.T_2.I.T_1 $
- $T_4$ *frame* transformation from camera frame $ T^W_{C_D} = T_3.T_2.I.T_1.T_4 $
- $T_5$ *frame* transformation from camera frame $ T^W_{C_E} = T_3.T_2.I.T_1.T_4.T_5 $
<br>
As $C_E$ is final camera frame. Finally, $T_C^W = T_3.T_2.T_1.T_4.T_5$

Now,
$$
\begin{align*}
    T^C_W  & = (T_C^W)^{-1} \\ 
           & = T_5^{-1}.T_4^{-1}.T_1^{-1}.T_2^{-1}.T_3^{-1} 
\end{align*}
$$

Camera to world *frame* transformation matrix ($T^C_W$),
$$
           T^C_W = T_5^{-1}.T_4^{-1}.T_1^{-1}.T_2^{-1}.T_3^{-1} 
$$
    


# Visualise the Data

Point clouds are a collection of points that represent a 3D shape or feature. Each point has its own set of X, Y and Z coordinates and in some cases additional attributes. A popular way to obtain this is by photogrammetry, though here we will use LiDAR data.

LiDAR is a remote sensing process which collects measurements used to create 3D models and maps of objects and environments. Using ultraviolet, visible, or near-infrared light, LiDAR gauges spatial relationships and shapes by measuring the time it takes for signals to bounce off objects and return to the scanner.

1. Download the data from [here](https://iiitaphyd-my.sharepoint.com/:f:/g/personal/venkata_surya_students_iiit_ac_in/EnYAMaTVIhJItzKYqtahE30BRKB6p6UfHN3TyJzvo6Mw0g?e=PegWds). It contains the LIDAR sensor output and odometry information per frame.

    The .bin files contain the 3D point cloud captured by the LIDAR in this format - x, y, z, and reflectance. 

    The odometry information is given in the `odometry.txt` file, which is a 12 element vector. Reshape each of the first 77 rows to a 3x4 matrix to obtain the pose.
    

2. Obtain the point cloud from this and visualise for 1-2 frames.

In [19]:
getPointCloudObj = lambda pcd : o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pcd))
pcd_l = []
for i in range(77):
    raw_pcd = np.fromfile(f'./data/data/LiDAR/0000{i + 10}.bin', dtype=np.float32)
    pcd = raw_pcd.reshape((-1,4))[::10, :3]
    pcd_l.append(getPointCloudObj(pcd))

In [20]:
o3d.visualization.draw_geometries([pcd_l[1]])

In [4]:
raw_poses = np.loadtxt('data/data/odometry.txt')
poses = raw_poses[:77].reshape(-1, 3,4)
poses = np.hstack((poses, np.array([[[0,0,0,1]]] * poses.shape[0])))

# Transform 

The point cloud obtained is with respect to the LiDAR frame. The poses however, are in the camera frame. If we want to combine the point clouds from various frames, we need to bring them to the camera frame. 

1. Refer to the image below and apply the required transformation to the point cloud. 

2. Then, register all point clouds into a common reference frame and visualise it (Open3D). It is helpful to use homogeneous coordinates to keep track of the different frames.

3. Write a function to transform the registered point cloud from the world to the $i^{th}$ camera frame, wherein $i$ is the input to the function.

4. \[Bonus\] Move around in the registered point cloud using arrow keys like you would do in a game. For this you will have to regularly transform the entire registered world to your current camera frame and visualize repeatedly. You may choose to avoid visualizing points that are behind the camera in this case as they are not visible from the scene. You may also visualize points at a max depth to make the process easier.

![](./img/transform.png)

In [21]:
import copy

In [22]:
# 1.
# Convention
# LiDAR -> l
# Camera -> ci
# World -> w
# Tba -> T_a^b

T_cil = np.array([[0,-1,0, 0], [0,0,-1 ,0 ],[1,0,0, 0],[0,0,0,1]]) # T_cil = Rz(90)*Ry(-90)
pcd_ci = []

for _pcd in pcd_l:
    pcd = copy.deepcopy(_pcd)
    pcd.transform(T_cil)
    pcd_ci.append(pcd)

# pcd_ci is pcd in camera co-ordinates in app the frame

In [26]:
# 2.
pcd_w_l= []
pcd_w = o3d.geometry.PointCloud()
# frame_n = 0
for pose, _pcd in zip(poses, pcd_ci):
    pcd = copy.deepcopy(_pcd)
    pcd.transform(pose)
    pcd_w += pcd
    pcd_w_l.append(pcd)

In [27]:
# pcd_w is total registered point cloud
o3d.visualization.draw_geometries([pcd_w])

In [28]:
# 3.
def transform_to_i(i):
    pcd = copy.deepcopy(pcd_w)
    # T_W^C = T_C^W-1
    T_ciw = np.linalg.inv(poses[i])
    pcd.transform(T_ciw)
    
    return pcd

In [29]:
# 4.
# press D to initialize then
# A for prev
# D for next

i = 0
geometery = copy.deepcopy(transform_to_i(0))
def moveNext(vis):
    global i
    control = vis.get_view_control()
    control.set_up([0,-1,0])
    control.set_lookat([0,0,-1])
    control.set_front([0,0,-1])
    control.set_zoom(0.01)
    if i == 76:
        return
    i+= 1
    pcd = np.asarray(transform_to_i(i).points)
    geometery.points = o3d.utility.Vector3dVector(pcd[pcd[:,2] >= 0])
    vis.update_geometry(geometery)
    return False
def moveBack(vis):
    global i
    control = vis.get_view_control()
    control.set_up([0,-1,0])
    control.set_lookat([0,0,-1])
    control.set_front([0,0,-1])
    control.set_zoom(0.01)
    if i == 0:
        return
    i-= 1
    pcd = np.asarray(transform_to_i(i).points)
    geometery.points = o3d.utility.Vector3dVector(pcd[pcd[:,2] >= 0])
    vis.update_geometry(geometery)
    return False

callbacks = {
    ord("D"): moveNext,
    ord("A"): moveBack
}

o3d.visualization.draw_geometries_with_key_callbacks([geometery], callbacks)

# Occupancy Map

Occupancy grid maps are discrete fine grain grid maps. These maps can be either 2-D or 3-D. Each cell in the occupancy grid map contains information on the physical objects present in the corresponding space. Since these maps shed light on what parts of the environment are occupied, and what is not, they are really useful for path planning and navigation.

Occupancy grid maps are probabilistic in nature due to noisy measurements. Each cell can have three states: Occupied, unoccupied, and unknown. For the purpose of this assignment, you can ignore the unknown and work in a binary setting where 1 is occupied and 0 is unoccupied.

1. The task here is to create an occupancy map for each LiDAR scan. You do not need to apply bayesian update rules here, just keep it simple. 

2. Now, using the *registered* point cloud, generate occupancy maps for each frame. What difference do you expect to see between the two methods?

You can mark a cell as occupied based on a threshold of how many different z values are there for a particular (x,y) cell.

In [30]:
# 1.
frame = 0
for pcd in pcd_l:
    points = copy.deepcopy(np.asarray(pcd.points))
    # points[:,1] = points[:,2]
    points = points[:,:2]
    points -= points.min()
    points = np.round(points).astype('int')
    h = points[:,1].max() + 10
    w = points[:,0].max() + 10
    occ = np.zeros((h,w))
    for point in points:
        occ[point[1], point[0]] += 1
    image = np.zeros(occ.shape)
    image[occ > 3] = 255
    cv2.imwrite('results/LiDAR occupancy/' + str(frame) + '.png', image)
    frame += 1
    

In [31]:
# 2.
frame = 0
for pose in poses:
    points = copy.deepcopy(pcd_w)
    points.transform(np.linalg.inv(pose))
    points = np.asarray(points.points)
    points[:,1] = points[:,2]
    points = points[:,:2]
    points -= points.min()
    points = np.round(points).astype('int')
    h = points[:,1].max() + 10
    w = points[:,0].max() + 10
    occ = np.zeros((h,w))
    for point in points:
        occ[point[1], point[0]] += 1
    image = np.zeros(occ.shape)
    image[occ > occ.max() * 2 / 3 ] = 255
    cv2.imwrite('results/Total occupancy/' + str(frame) + '.png', image)
    frame += 1

Occupancy map in part 2 is more robust and more dense due to presence of more data points